In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
sp_data=pd.read_html(wiki_page)
ticker_df=sp_data[0]
ticker_options=ticker_df['Symbol']

In [6]:
ticker_options[0]

'MMM'

In [16]:
ticker_options.shape

(503,)

In [9]:
def level(x):
    if x==0.0:
        return 'No Controversy'
    elif x==1.0:
        return 'Little Controversy'
    elif x==2.0:
        return 'Moderate Controversy'
    elif x==3.0:
        return 'Relatively High Controversy'
    else:
        return 'Severe Controversy'

In [10]:
def web_scraper(ticker):
    elements = []
    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36","Accept-Language":"en-US,en;q=0.9"}
    web_data = requests.get('https://finance.yahoo.com/quote/'+ticker+'/sustainability?p='+ticker, headers=headers).text
    soup=BeautifulSoup(web_data, "html.parser")
    esg_score = soup.find('div', class_='Fz(36px) Fw(600) D(ib) Mend(5px)')
    datapoint = esg_score.text
    controversy_score = soup.find('div', class_='D(ib) Fz(36px) Fw(500)')
    controversy_datapoint = controversy_score.text
    scores = soup.find_all('div', class_='D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)')
    for score in scores:
        elements.append(score.text)

    df=pd.DataFrame({'Total ESG Score':datapoint,
                     'Environmental Score':elements[0],
                     'Social Score':elements[1],
                     'Governance Score':elements[2],
                     'Controversy Score':controversy_datapoint}, index=[ticker])
    df=df.astype('float')
    df['Controversy Assessment']=df.apply(lambda x: level(x['Controversy Score']),axis=1)
    return df

In [23]:
new_df=web_scraper(ticker_options[0])

In [24]:
new_df

,Total ESG Score,Environmental Score,Social Score,Governance Score,Controversy Score,Controversy Assessment
MMM,34.0,12.3,13.6,7.7,3.0,Relatively High Controversy


In [26]:
new_df2=web_scraper(ticker_options[1])

In [27]:
new_df2

,Total ESG Score,Environmental Score,Social Score,Governance Score,Controversy Score,Controversy Assessment
AOS,24.0,7.1,10.8,6.2,0.0,No Controversy


In [25]:
df2=pd.DataFrame()

In [28]:
pd.concat([new_df,new_df2])

,Total ESG Score,Environmental Score,Social Score,Governance Score,Controversy Score,Controversy Assessment
MMM,34.0,12.3,13.6,7.7,3.0,Relatively High Controversy
AOS,24.0,7.1,10.8,6.2,0.0,No Controversy


In [38]:
cand=pd.DataFrame()
for n in range(3):
    cand=cand.append(web_scraper(ticker_options[n]))

C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1791816294.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cand=cand.append(web_scraper(ticker_options[n]))
C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1791816294.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cand=cand.append(web_scraper(ticker_options[n]))
C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1791816294.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cand=cand.append(web_scraper(ticker_options[n]))


In [39]:
cand

,Total ESG Score,Environmental Score,Social Score,Governance Score,Controversy Score,Controversy Assessment
MMM,34.0,12.3,13.6,7.7,3.0,Relatively High Controversy
AOS,24.0,7.1,10.8,6.2,0.0,No Controversy
ABT,25.0,3.0,13.6,8.4,3.0,Relatively High Controversy


In [42]:
cand.to_excel("S&Pesgtest.xlsx",encoding='utf8', index=False)

In [47]:
candall=pd.DataFrame()
for n in range(503):
    try:
        candall=candall.append(web_scraper(ticker_options[n]))
    except:
        pass

C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1002790452.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  candall=candall.append(web_scraper(ticker_options[n]))
C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1002790452.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  candall=candall.append(web_scraper(ticker_options[n]))
C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1002790452.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  candall=candall.append(web_scraper(ticker_options[n]))
C:\Users\hejie\AppData\Local\Temp\ipykernel_16660\1002790452.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  candall=candall.append(web_

In [48]:
candall.to_excel("S&P_ESG_Scores.xlsx",encoding='utf8', index=False)